# Panda Example

In [1]:
import sys
sys.path.append('lib')

import scipy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time, os
from IPython.display import clear_output
from IPython.core import display

import pybullet as p
import pybullet_data
import pinocchio as pin

from pb_utils.kine import get_joint_limits, check_joint_limits
from pb_utils.visualize import set_q, vis_traj, create_primitives
from pin_utils.pin_utils import computeJacobian, computePose

from residuals import ResidualFrameTranslationFloatingBase, ResidualFrameRotationSE3FloatingBase, ResidualBound, \
                        ResidualPosture, ResidualSum, ResidualStructure, CostProjector, clip_bounds
from motion_planning import sampler
from panda_config import pb_joint_indices
from functools import partial
from motion_planning import cRRT
DATA_PATH = '../../data'
robot_urdf = DATA_PATH + '/urdf/panda_arm.urdf'

%load_ext autoreload
%autoreload 2

np.set_printoptions(precision=4, suppress=True)

### Setup Pybullet

In [2]:
physics_client_id = p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)

In [3]:
p.resetSimulation()

### Setup Robot & environment

In [4]:
robot_id = p.loadURDF(fileName=robot_urdf)
dof = p.getNumJoints(robot_id)
pb_joint_indices = np.arange(7)
joint_limits = get_joint_limits(robot_id,pb_joint_indices)
mean_pose = 0.5*(joint_limits[0]+joint_limits[1])

plane_id = p.loadURDF('plane.urdf')
p.resetBasePositionAndOrientation(plane_id, (0,0,-.5), (0,0,0,1))

table_square_id = p.loadURDF('table_square/table_square.urdf')
p.resetBasePositionAndOrientation(table_square_id, (0.,0,-0.64), (0, 0, 0.7071068, 0.7071068))

table_id = p.loadURDF('table/table.urdf')
p.resetBasePositionAndOrientation(table_id, (.7,0,-0.5), (0, 0, 0.7071068, 0.7071068))

shelf_urdf = DATA_PATH + '/urdf/bookshelf_simple_collision.urdf'
shelf_id = p.loadURDF(fileName=shelf_urdf)
p.resetBasePositionAndOrientation(shelf_id, (-0.6,0.6,-0.5), (0, 0, 0, 1.))

#for visualizing the desired target
_,_,ball_id = create_primitives(radius=0.05)

### Load model in pinocchio

In [5]:
rmodel = pin.buildModelFromUrdf(robot_urdf)
rdata = rmodel.createData()

pin_frame_names = [f.name for f in rmodel.frames]
ee_frame_id = rmodel.getFrameId('panda_hand2')

#### Define standard functions

In [6]:
computeJacobian_std = partial(computeJacobian, rmodel, rdata, ee_frame_id)
computePose_std = partial(computePose, rmodel, rdata, ee_frame_id)
set_q_std = partial(set_q,robot_id=robot_id, joint_indices=pb_joint_indices)

***

## Using Cost Model for projection

In [7]:
from pb_utils.transform import w2mat

In [8]:
rotation_ik = np.eye(3) #desired orientation matrix
rotation_ik_shelf = w2mat((0,0,np.pi/2))
rotation_ik_table = w2mat((0,0,0))
pose_cost = ResidualFrameTranslationFloatingBase(rmodel, rdata, np.zeros(3), ee_frame_id, weight=np.array([1.,1.,1.]))
ori_cost = ResidualFrameRotationSE3FloatingBase(rmodel, rdata, rotation_ik, ee_frame_id, weight=np.array([1.,1.,0.]))
ori_cost_shelf = ResidualFrameRotationSE3FloatingBase(rmodel, rdata, rotation_ik_shelf, ee_frame_id, weight=np.array([1.,1.,1.]))
ori_cost_table = ResidualFrameRotationSE3FloatingBase(rmodel, rdata, rotation_ik_table, ee_frame_id, weight=np.array([1.,1.,1.]))
bound_cost = ResidualBound(joint_limits, 1e-4)
posture_cost = ResidualPosture(rmodel, rdata, mean_pose)

In [238]:
#for projection
cost_sum = ResidualSum(rmodel, rdata)
cost_sum.addCost(ori_cost, 20., 'ori_cost', 1e-4)
cost_sum.addCost(bound_cost, 20., 'joint_limit', 1e-3)

#for IK with horizontal gripper constraint
cost_sum_ik = ResidualSum(rmodel, rdata)
cost_sum_ik.addCost(pose_cost,50., 'pose_cost', 1e-3)
cost_sum_ik.addCost(ori_cost, 30., 'ori_cost', 1e-3)
cost_sum_ik.addCost(bound_cost, 30., 'joint_limit', 1e-3)

#for IK with gripper facing the shelf
cost_sum_ik_shelf = ResidualSum(rmodel, rdata)
cost_sum_ik_shelf.addCost(pose_cost,50., 'pose_cost', 1e-3)
cost_sum_ik_shelf.addCost(ori_cost_shelf, 30., 'ori_cost', 1e-3)
cost_sum_ik_shelf.addCost(bound_cost, 30., 'joint_limit', 1e-3)

#for IK with gripper facing the table
cost_sum_ik_table = ResidualSum(rmodel, rdata)
cost_sum_ik_table.addCost(pose_cost,50., 'pose_cost', 1e-3)
cost_sum_ik_table.addCost(ori_cost_table, 30., 'ori_cost', 1e-3)
cost_sum_ik_table.addCost(bound_cost, 30., 'joint_limit', 1e-3)

#secondary task: posture regularization
cost_sum2 = ResidualSum(rmodel, rdata)
cost_sum2.addCost(posture_cost, 1. , 'posture', 1e3)

robot_projector = CostProjector(cost_sum, rmodel, rdata, cost2 = cost_sum2,bounds = joint_limits)
robot_ik_solver = CostProjector(cost_sum_ik, rmodel, rdata, bounds = joint_limits)
robot_ik_solver_shelf = CostProjector(cost_sum_ik_shelf, rmodel, rdata, cost2 = cost_sum2, bounds = joint_limits)
robot_ik_solver_table = CostProjector(cost_sum_ik_table, rmodel, rdata, cost2 = cost_sum2, bounds = joint_limits)

### Define the sampler

In [136]:
from motion_planning import sampler, col_checker, interpolator

In [137]:
workspace_area = np.array([[-0.6, -0.6 , -0.1], [0.6, 0.6, 1.0]])
#workspace_area = np.array([[-0.8, -0.8 , 0.1], [0.8, 0.8, 1.0]])
target_sampler = sampler(workspace_area)
rob_simple_sampler = sampler(joint_limits)
rob_col_checker = col_checker(robot_id, pb_joint_indices, [ plane_id, shelf_id, table_id, table_square_id])
rob_interpolator = interpolator()

#### Try the projection operator

In [138]:
q = rob_simple_sampler.sample().flatten()
res = robot_projector.project(q)
q, success, func_calls = res['q'], res['stat'], res['nfev']
set_q_std(q)

In [139]:
is_collide = True
while is_collide is True:
    q = rob_simple_sampler.sample().flatten()
    res = robot_projector.project(q)
    q, success, func_calls = res['q'], res['stat'], res['nfev']
    is_collide = rob_col_checker.check_collision(q)
set_q_std(q.flatten())

#### Compare IK

In [140]:
ik_workspace_area = np.array([[0.4, -0.5 , 0.5], [0.7, 0.5, .8]])
ik_target_sampler = sampler(ik_workspace_area)
goal_poses = ik_target_sampler.sample(1000)
for i in range(len(goal_poses)):
    goal_poses[i] = clip_bounds(goal_poses[i], ik_workspace_area)
# qs = rob_gan_sampler.sample(N=1000, _targets=goal_poses, var = 0.01)
qs = rob_simple_sampler.sample(N=1000)

#### Try IK

In [141]:
idx = np.random.randint(1000)
goal_pos = goal_poses[idx]
if np.linalg.norm(goal_pos[:2]) < 0.3:
    print('Likely to fail')

if np.linalg.norm(goal_pos) > 0.8:
    print('Likely to fail')
    
p.resetBasePositionAndOrientation(ball_id, goal_pos, np.array([0,0,0,1]))

robot_ik_solver.cost.costs['pose_cost'].cost.desired_pose = goal_pos
q = qs[idx]
#q = rob_simple_sampler.sample().flatten()
set_q_std(q)

res = robot_ik_solver.project(q, maxiter = 100)
print(res['stat'])
set_q_std(res['q'])

Likely to fail
False


## Setup different RRTs

In [142]:
class simple_goal():
    def __init__(self, sampler, projector_pose, x_goal):
        self.projector_pose = projector_pose
        self.sampler = sampler
        self.x_goal = x_goal
        
    def check(self, q):
        pos, ori = computePose_std(q)
        if np.linalg.norm(pos - x_goal) < self.g_tol:
            return True
        return False
    
    def sample(self):
        sample = self.sampler.sample()
        self.projector_pose.cost.costs['pose_cost'].cost.desired_pose[:3] = self.x_goal
        res = self.projector_pose.project(sample.flatten())
        proj_sample, cost, success = res['q'], res['feval'], res['stat']
        return proj_sample, success

In [143]:
def sample_valid_q():
    is_collide = True
    status = False
    while is_collide is True or status is False:
        q = rob_simple_sampler.sample().flatten()
        res = robot_projector.project(q)
        q, status, func_calls_  = res['q'], res['stat'], res['nfev']
        is_collide = rob_col_checker.check_collision(q.flatten())
        
    return q.flatten()

In [144]:
def sample_goal_states(N, goal):
    q_goals = []

    for i in range(N):
        while(1):
            q_goal, success = goal.sample()
            if success and rob_col_checker.check_collision(q_goal) is False:
                q_goals += [q_goal]
                break
                
    return q_goals

In [146]:
standard_rrt =cRRT(7, rob_simple_sampler,  rob_col_checker, rob_interpolator, robot_projector)

### Perform N Tasks to compare the runtime of RRTs

In [147]:
x_goal_bottom_left = np.array([-0.4, 0.65, 0.2])
x_goal_top_left = np.array([-0.4, 0.65, 0.54])
x_goal_bottom_right = np.array([-0.02, 0.65, 0.2])
x_goal_top_right = np.array([-0.02, 0.65, 0.54])

x_goals = []
x_goals += [x_goal_bottom_left]
x_goals += [x_goal_top_left]
x_goals += [x_goal_bottom_right]
x_goals += [x_goal_top_right]

x_inits = np.array([[0.5, -0.25, 0.2],
                    [0.5, 0.25, 0.2],
                    [0.7, -0.25, 0.2],
                    [0.7, 0.25, 0.2]])

N = 2
p_goal = 0.5
p_random_hybrid = 0.2
p_random_gan = 0.0
p_random_uniform = 1.0
max_extension_steps = 500

In [148]:
import time

In [273]:
rob_simple_init = simple_goal(rob_simple_sampler, robot_ik_solver_table, x_inits[np.random.randint(4)])
q_init = sample_goal_states(1, rob_simple_init)[0]

rand_idx = np.random.randint(4)
rob_simple_goal = simple_goal(rob_simple_sampler, robot_ik_solver_shelf, x_goals[rand_idx])
q_goals = sample_goal_states(10, rob_simple_goal)
print('Uniform...')    
traj, proj_iter, ext_steps, success, n_retry, t , path = standard_rrt.plan(q_init, q_goals,max_extension_steps=500) 

Uniform...
Solution is found!
Solution found!


In [274]:
short_path = standard_rrt.shortcut_path(path, step_length=0.05)

short_traj = standard_rrt.interpolate_traj(short_path, step_length=0.05)

In [275]:
vis_traj(short_traj, set_q_std, 0.02)